# 🔌 MCP + OpenAI Agents SDK — Hands-On Tutorial

This notebook teaches you the **Model Context Protocol (MCP)** from scratch and shows how to use it with the **OpenAI Agents SDK** to build multi-agent systems that interact with external tools and data.

| Section | What you'll learn |
|---|---|
| **1** | What is MCP? — The mental model |
| **2** | Installation & setup |
| **3** | Building your first MCP server (with FastMCP) |
| **4** | Connecting an OpenAI Agent to an MCP server |
| **5** | MCP with function tools — combining both |
| **6** | Multi-agent + MCP architecture |
| **7** | Full workflow: Excel file upload → multi-agent processing → file returned |
| **8** | Proposed project structure for production |

---

### What is MCP in one sentence?

> **MCP is a universal "USB-C port" for AI**: a standard protocol that lets any AI application discover and call tools on any MCP server, regardless of language, framework, or transport.

### Why does it matter?

Without MCP, every AI app reinvents tool integration. With MCP:
- You build a tool server **once**, and any MCP-compatible AI host can use it.
- You get a standardized way to expose **tools**, **resources**, and **prompts**.
- You can mix local servers (stdin/stdout) and remote servers (HTTP) freely.

> **Docs:**  
> - MCP Spec: https://modelcontextprotocol.io  
> - OpenAI Agents SDK + MCP: https://openai.github.io/openai-agents-python/mcp/

---
## 1 · What is MCP? — The Mental Model

### The Architecture

```
┌─────────────────────────────────────────────────────┐
│                    MCP HOST                         │
│  (Your AI app — e.g. OpenAI Agents SDK, Claude,     │
│   VS Code Copilot, or your own Python script)       │
│                                                     │
│   ┌────────────┐  ┌────────────┐  ┌────────────┐   │
│   │ MCP Client │  │ MCP Client │  │ MCP Client │   │
│   │     #1     │  │     #2     │  │     #3     │   │
│   └─────┬──────┘  └─────┬──────┘  └─────┬──────┘   │
└─────────┼───────────────┼───────────────┼───────────┘
          │               │               │
    ┌─────▼──────┐  ┌─────▼──────┐  ┌─────▼──────┐
    │ MCP Server │  │ MCP Server │  │ MCP Server │
    │ Filesystem │  │  Database  │  │  Excel     │
    │            │  │            │  │ Processor  │
    └────────────┘  └────────────┘  └────────────┘
```

### Key Participants

| Participant | Role | Example |
|---|---|---|
| **MCP Host** | The AI application that manages clients | Your Python script using the Agents SDK |
| **MCP Client** | Maintains a connection to one MCP server | Created automatically by the SDK |
| **MCP Server** | Exposes tools, resources, and prompts | A Python process that reads/writes Excel files |

### The 3 Primitives an MCP Server Can Expose

| Primitive | What it is | Example |
|---|---|---|
| **Tools** | Functions the LLM can call | `read_excel_file()`, `write_cell()`, `query_database()` |
| **Resources** | Read-only data the LLM can access | File contents, DB schemas, API docs |
| **Prompts** | Reusable prompt templates | "Analyze this spreadsheet for errors" |

### Transport Options

| Transport | When to use | SDK class |
|---|---|---|
| **stdio** | Local dev, subprocess-based servers | `MCPServerStdio` |
| **Streamable HTTP** | Remote/production servers | `MCPServerStreamableHttp` |
| **SSE** (deprecated) | Legacy servers only | `MCPServerSse` |
| **Hosted** | OpenAI-managed, zero infra | `HostedMCPTool` |

### How a Tool Call Flows Through MCP

```
User: "What's in my spreadsheet?"
  │
  ▼
Agent (LLM) sees available tools via tools/list
  │
  ▼
LLM decides to call: read_excel(path="data.xlsx")
  │
  ▼
SDK sends JSON-RPC → MCP Server → tools/call
  │
  ▼
MCP Server runs read_excel(), returns result
  │
  ▼
SDK sends result back to LLM
  │
  ▼
LLM formulates final answer for user
```

---
## 2 · Installation & Setup

In [ ]:
# Install dependencies
%pip install openai-agents "mcp[cli]" openpyxl pandas --quiet

In [ ]:
import os

# Set your OpenAI API key
# os.environ["OPENAI_API_KEY"] = "sk-..."

assert os.environ.get("OPENAI_API_KEY"), "⚠️  Please set OPENAI_API_KEY before continuing."
print("✅ API key is set.")

---
## 3 · Building Your First MCP Server (with FastMCP)

An MCP server is a standalone Python program that exposes tools. The `mcp` Python package provides **FastMCP** — a high-level helper that auto-generates tool schemas from type hints (just like `@function_tool` in the Agents SDK).

### How it works
1. You create a `FastMCP` instance.
2. You decorate functions with `@mcp.tool()` — these become tools the LLM can call.
3. You run the server (over stdio, HTTP, etc.).
4. An MCP client (like the Agents SDK) connects and discovers the tools automatically.

Let's write a simple MCP server that does math and string operations. We'll save it as a Python file so we can launch it as a subprocess.

In [ ]:
%%writefile ../mcp_servers/math_server.py
"""
A simple MCP server that exposes math and string tools.
This runs as a subprocess — the Agents SDK communicates with it over stdin/stdout.
"""
from mcp.server.fastmcp import FastMCP

# Create the MCP server
mcp = FastMCP("math-tools")


@mcp.tool()
def add(a: float, b: float) -> float:
    """Add two numbers together.

    Args:
        a: First number.
        b: Second number.
    """
    return a + b


@mcp.tool()
def multiply(a: float, b: float) -> float:
    """Multiply two numbers.

    Args:
        a: First number.
        b: Second number.
    """
    return a * b


@mcp.tool()
def reverse_string(text: str) -> str:
    """Reverse a string.

    Args:
        text: The text to reverse.
    """
    return text[::-1]


@mcp.tool()
def word_count(text: str) -> int:
    """Count the number of words in a text.

    Args:
        text: The text to count words in.
    """
    return len(text.split())


if __name__ == "__main__":
    # Run over stdio — the Agents SDK will spawn this process
    mcp.run(transport="stdio")

We just created a file at `../mcp_servers/math_server.py`. This server:

- Exposes 4 tools: `add`, `multiply`, `reverse_string`, `word_count`
- Runs over **stdio** (the SDK spawns it as a subprocess)
- Auto-generates JSON schemas from the Python type hints + docstrings

> **Key insight:** The MCP server is a standalone program. It knows nothing about the LLM or the Agents SDK. It just exposes tools via the MCP protocol.

---
## 4 · Connecting an OpenAI Agent to an MCP Server

Now the magic: we connect our agent to the MCP server. The agent will **automatically discover** all tools the server exposes.

### How the connection works

```python
from agents.mcp import MCPServerStdio

# This tells the SDK: "spawn this Python script and talk to it over stdin/stdout"
server = MCPServerStdio(
    name="Math Tools",
    params={
        "command": "python",
        "args": ["path/to/math_server.py"],
    },
)

# Give the server to an agent — it auto-discovers all tools!
agent = Agent(
    name="Assistant",
    mcp_servers=[server],  # ← this is the key
)
```

Let's try it:

In [ ]:
import asyncio
import sys
from pathlib import Path
from agents import Agent, Runner
from agents.mcp import MCPServerStdio

# Path to our MCP server
server_path = str(Path("../mcp_servers/math_server.py").resolve())
python_executable = sys.executable  # use the same Python that's running this notebook


async def run_with_mcp():
    # Create the MCP server connection (as a context manager)
    async with MCPServerStdio(
        name="Math Tools",
        params={
            "command": python_executable,
            "args": [server_path],
        },
    ) as server:
        # Create an agent that uses the MCP server
        agent = Agent(
            name="Math Assistant",
            instructions=(
                "You are a helpful assistant. Use the available tools to perform "
                "calculations and string operations. Always use tools rather than "
                "doing calculations yourself."
            ),
            mcp_servers=[server],
            model="gpt-4o-mini",
        )

        # Run the agent
        result = await Runner.run(agent, "What is 42 + 58? Also, reverse the word 'protocol'.")
        print("🤖 Response:", result.final_output)

        # Let's also see what tools were discovered
        tools = await server.list_tools()
        print("\n📋 Tools discovered from MCP server:")
        for tool in tools:
            print(f"   - {tool.name}: {tool.description}")


await run_with_mcp()

### What just happened?

1. The SDK spawned `math_server.py` as a subprocess.
2. It connected via stdin/stdout and called `tools/list` to discover all 4 tools.
3. The LLM saw those tools and decided to call `add(42, 58)` and `reverse_string("protocol")`.
4. The SDK routed those calls to the MCP server via `tools/call`.
5. The results came back, and the LLM composed its final answer.

> **Important:** The agent never "knew" the tool code. It only saw the name, description, and parameter schema — all auto-generated by FastMCP from your Python functions.

---
## 5 · MCP + Function Tools — Combining Both

You can use MCP servers **alongside** regular `@function_tool` tools on the same agent. This is powerful because:
- MCP tools come from external servers (reusable across projects).
- Function tools are defined inline (quick, project-specific logic).

The agent sees all tools from both sources and can call any of them.

In [ ]:
from agents import Agent, Runner, function_tool
from agents.mcp import MCPServerStdio
import sys
from pathlib import Path


# A regular function tool (inline, project-specific)
@function_tool
def get_current_date() -> str:
    """Get today's date."""
    from datetime import date
    return str(date.today())


@function_tool
def format_currency(amount: float, currency: str = "USD") -> str:
    """Format a number as currency.

    Args:
        amount: The amount to format.
        currency: Currency code (e.g. USD, EUR, NOK).
    """
    symbols = {"USD": "$", "EUR": "€", "NOK": "kr", "GBP": "£"}
    symbol = symbols.get(currency, currency)
    return f"{symbol}{amount:,.2f}"


server_path = str(Path("../mcp_servers/math_server.py").resolve())


async def run_combined():
    async with MCPServerStdio(
        name="Math Tools",
        params={"command": sys.executable, "args": [server_path]},
    ) as server:
        agent = Agent(
            name="Combined Assistant",
            instructions=(
                "You are a helpful assistant with access to math tools (via MCP), "
                "plus date and currency formatting tools. Use them as needed."
            ),
            mcp_servers=[server],             # MCP tools (add, multiply, etc.)
            tools=[get_current_date, format_currency],  # Regular function tools
            model="gpt-4o-mini",
        )

        result = await Runner.run(
            agent,
            "What's today's date? Calculate 1500 * 1.25 and format the result as NOK currency.",
        )
        print("🤖", result.final_output)


await run_combined()

---
## 6 · Multi-Agent + MCP Architecture

Now let's combine **multiple agents** with **MCP servers**. Each agent can have its own MCP servers, or they can share them.

### Pattern: Specialized agents with dedicated MCP servers

```
                    ┌──────────────┐
                    │   Triage     │
                    │   Agent      │
                    └──────┬───────┘
                           │ handoffs
              ┌────────────┼────────────┐
              ▼            ▼            ▼
     ┌────────────┐ ┌────────────┐ ┌────────────┐
     │  Data      │ │  Analysis  │ │  Report    │
     │  Agent     │ │  Agent     │ │  Agent     │
     └─────┬──────┘ └─────┬──────┘ └─────┬──────┘
           │              │              │
     ┌─────▼──────┐ ┌─────▼──────┐ ┌─────▼──────┐
     │ MCP Server │ │ MCP Server │ │ MCP Server │
     │ (File I/O) │ │ (Stats)    │ │ (Charting) │
     └────────────┘ └────────────┘ └────────────┘
```

Let's build a simulated version of this. We'll create an MCP server for file operations and connect multiple agents.

In [ ]:
%%writefile ../mcp_servers/file_tools_server.py
"""
MCP server that provides file reading/writing tools.
Used by agents that need to interact with the filesystem.
"""
import json
import os
from pathlib import Path
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("file-tools")

# Define a safe working directory
WORK_DIR = Path(os.environ.get("MCP_WORK_DIR", "/tmp/mcp_workdir"))
WORK_DIR.mkdir(parents=True, exist_ok=True)


@mcp.tool()
def list_files(directory: str = ".") -> str:
    """List files in the working directory.

    Args:
        directory: Subdirectory to list (relative to work dir). Defaults to root.
    """
    target = WORK_DIR / directory
    if not target.exists():
        return f"Directory not found: {directory}"
    files = []
    for item in sorted(target.iterdir()):
        kind = "📁" if item.is_dir() else "📄"
        size = item.stat().st_size if item.is_file() else 0
        files.append(f"{kind} {item.name} ({size} bytes)")
    return "\n".join(files) if files else "(empty directory)"


@mcp.tool()
def read_text_file(filename: str) -> str:
    """Read a text file from the working directory.

    Args:
        filename: Name of the file to read.
    """
    path = WORK_DIR / filename
    if not path.exists():
        return f"File not found: {filename}"
    return path.read_text()


@mcp.tool()
def write_text_file(filename: str, content: str) -> str:
    """Write content to a text file in the working directory.

    Args:
        filename: Name of the file to write.
        content: The content to write.
    """
    path = WORK_DIR / filename
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(content)
    return f"✅ Wrote {len(content)} characters to {filename}"


if __name__ == "__main__":
    mcp.run(transport="stdio")

In [ ]:
import asyncio
import sys
from pathlib import Path
from agents import Agent, Runner
from agents.mcp import MCPServerStdio

math_server_path = str(Path("../mcp_servers/math_server.py").resolve())
file_server_path = str(Path("../mcp_servers/file_tools_server.py").resolve())


async def run_multi_agent_mcp():
    # Start both MCP servers
    async with MCPServerStdio(
        name="Math Tools",
        params={"command": sys.executable, "args": [math_server_path]},
    ) as math_server, MCPServerStdio(
        name="File Tools",
        params={"command": sys.executable, "args": [file_server_path]},
    ) as file_server:

        # Data agent: reads/writes files
        data_agent = Agent(
            name="Data Agent",
            instructions="You manage files. Use your file tools to read, write, and list files.",
            mcp_servers=[file_server],
            model="gpt-4o-mini",
        )

        # Analysis agent: does calculations
        analysis_agent = Agent(
            name="Analysis Agent",
            instructions="You perform calculations and analysis. Use your math tools.",
            mcp_servers=[math_server],
            model="gpt-4o-mini",
        )

        # Orchestrator uses both as tools
        orchestrator = Agent(
            name="Orchestrator",
            instructions=(
                "You coordinate between a data agent and an analysis agent.\n"
                "Use the data agent to read/write files and the analysis agent for calculations.\n"
                "Combine their results to answer the user's question."
            ),
            tools=[
                data_agent.as_tool(
                    tool_name="data_agent",
                    tool_description="Read, write, and list files in the working directory.",
                ),
                analysis_agent.as_tool(
                    tool_name="analysis_agent",
                    tool_description="Perform math calculations (add, multiply, etc.).",
                ),
            ],
            model="gpt-4o-mini",
        )

        result = await Runner.run(
            orchestrator,
            "Calculate 250 * 4, then save the result to a file called 'result.txt'. "
            "After that, list the files to confirm it was saved.",
        )
        print("🤖 Orchestrator:", result.final_output)


await run_multi_agent_mcp()

### What we just demonstrated

- **Two separate MCP servers** running simultaneously (math + file I/O).
- **Two specialized agents**, each connected to their own MCP server.
- **One orchestrator** that calls both agents as tools.
- The orchestrator doesn't need to know about MCP at all — it just calls the sub-agents.

---
## 7 · Full Workflow: Excel File → Multi-Agent Processing → Updated File

Now the main event. Let's build a complete workflow where:

1. **User uploads an Excel file** (we'll create a sample one).
2. **An MCP server** provides tools to read/write Excel files.
3. **Multiple agents** collaborate to analyze and modify the data.
4. **The updated file** is saved back to disk.

### The Agent Architecture

```
User: "Analyze this Excel file and fix the data"
  │
  ▼
┌──────────────────────────────────────────────┐
│           Orchestrator Agent                 │
│  Coordinates the entire workflow             │
│                                              │
│  Uses sub-agents as tools:                   │
│  ┌──────────────┐  ┌───────────────────────┐ │
│  │ Reader Agent  │  │ Analyst Agent         │ │
│  │ (reads Excel) │  │ (analyzes & suggests) │ │
│  └──────┬───────┘  └───────────────────────┘ │
│         │                                    │
│  ┌──────▼───────┐  ┌───────────────────────┐ │
│  │ Writer Agent  │  │ Summary Agent         │ │
│  │ (writes Excel)│  │ (summarizes changes)  │ │
│  └──────┬───────┘  └───────────────────────┘ │
└─────────┼────────────────────────────────────┘
          │
    ┌─────▼────────┐
    │  MCP Server   │
    │  (Excel I/O)  │
    └──────────────┘
```

### Step 1: Create a sample Excel file

In [ ]:
import pandas as pd
from pathlib import Path

# Create a sample Excel file with some intentional issues
data = {
    "Employee": ["Alice", "Bob", "Carol", "David", "Eve", "Frank"],
    "Department": ["Engineering", "Sales", "Engineering", "Marketing", "Sales", "Engineering"],
    "Hours_Worked": [160, 145, 170, 155, 0, 180],        # Eve has 0 hours — suspicious
    "Hourly_Rate": [75.0, 55.0, 80.0, 60.0, 55.0, 72.0],
    "Bonus": [500, 300, 600, 0, 250, 400],                # David has 0 bonus — check
    "Total_Pay": [12500, 8275, 14200, 9300, 250, 13360],  # Some are wrong on purpose!
}

df = pd.DataFrame(data)

# Save to the working directory
work_dir = Path("/tmp/mcp_workdir")
work_dir.mkdir(parents=True, exist_ok=True)
excel_path = work_dir / "employee_payroll.xlsx"
df.to_excel(excel_path, index=False)

print("✅ Sample Excel file created at:", excel_path)
print()
print(df.to_string(index=False))
print()
print("📌 Note: Total_Pay should be (Hours_Worked × Hourly_Rate) + Bonus")
print("   Some values are intentionally wrong so the agents can find and fix them!")

### Step 2: Create the Excel MCP Server

In [ ]:
%%writefile ../mcp_servers/excel_server.py
"""
MCP server for Excel file operations.
Provides tools to read, analyze, and modify Excel files.
"""
import json
import os
from pathlib import Path

import openpyxl
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("excel-tools")

WORK_DIR = Path(os.environ.get("MCP_WORK_DIR", "/tmp/mcp_workdir"))


@mcp.tool()
def read_excel(filename: str) -> str:
    """Read an Excel file and return its contents as a formatted table.

    Args:
        filename: Name of the Excel file in the working directory.
    """
    path = WORK_DIR / filename
    if not path.exists():
        return f"File not found: {filename}"

    wb = openpyxl.load_workbook(path)
    ws = wb.active

    rows = []
    for row in ws.iter_rows(values_only=True):
        rows.append([str(cell) if cell is not None else "" for cell in row])

    if not rows:
        return "(empty spreadsheet)"

    # Format as a readable table
    header = rows[0]
    lines = [" | ".join(header)]
    lines.append("-" * len(lines[0]))
    for row in rows[1:]:
        lines.append(" | ".join(row))

    return f"Sheet: {ws.title}\nRows: {len(rows) - 1} (excluding header)\n\n" + "\n".join(lines)


@mcp.tool()
def get_cell_value(filename: str, cell_reference: str) -> str:
    """Get the value of a specific cell.

    Args:
        filename: Name of the Excel file.
        cell_reference: Cell reference like 'A1', 'B3', etc.
    """
    path = WORK_DIR / filename
    if not path.exists():
        return f"File not found: {filename}"

    wb = openpyxl.load_workbook(path)
    ws = wb.active
    value = ws[cell_reference].value
    return f"Cell {cell_reference} = {value}"


@mcp.tool()
def update_cell(filename: str, cell_reference: str, value: str) -> str:
    """Update a specific cell in an Excel file.

    Args:
        filename: Name of the Excel file.
        cell_reference: Cell reference like 'A1', 'B3', etc.
        value: The new value for the cell (will auto-detect numbers).
    """
    path = WORK_DIR / filename
    if not path.exists():
        return f"File not found: {filename}"

    wb = openpyxl.load_workbook(path)
    ws = wb.active

    # Try to convert to number
    try:
        numeric_value = float(value)
        if numeric_value == int(numeric_value):
            numeric_value = int(numeric_value)
        ws[cell_reference] = numeric_value
    except ValueError:
        ws[cell_reference] = value

    wb.save(path)
    return f"✅ Updated {cell_reference} to {value} in {filename}"


@mcp.tool()
def get_column_stats(filename: str, column_letter: str) -> str:
    """Get basic statistics for a numeric column.

    Args:
        filename: Name of the Excel file.
        column_letter: The column letter (e.g. 'C', 'D').
    """
    path = WORK_DIR / filename
    if not path.exists():
        return f"File not found: {filename}"

    wb = openpyxl.load_workbook(path)
    ws = wb.active

    values = []
    header = None
    for i, row in enumerate(ws.iter_rows(min_col=ord(column_letter) - 64,
                                         max_col=ord(column_letter) - 64,
                                         values_only=True), 1):
        if i == 1:
            header = row[0]
            continue
        if row[0] is not None:
            try:
                values.append(float(row[0]))
            except (ValueError, TypeError):
                pass

    if not values:
        return f"No numeric values found in column {column_letter}"

    return json.dumps({
        "column": column_letter,
        "header": header,
        "count": len(values),
        "sum": sum(values),
        "mean": sum(values) / len(values),
        "min": min(values),
        "max": max(values),
    }, indent=2)


@mcp.tool()
def add_row(filename: str, values_json: str) -> str:
    """Add a new row at the bottom of the spreadsheet.

    Args:
        filename: Name of the Excel file.
        values_json: JSON array of values for the new row, e.g. '["Name", "Dept", 100]'.
    """
    path = WORK_DIR / filename
    if not path.exists():
        return f"File not found: {filename}"

    wb = openpyxl.load_workbook(path)
    ws = wb.active

    row_values = json.loads(values_json)
    ws.append(row_values)
    wb.save(path)

    return f"✅ Added row with {len(row_values)} values to {filename}"


if __name__ == "__main__":
    mcp.run(transport="stdio")

### Step 3: Build the Multi-Agent Excel Processing Workflow

In [ ]:
import sys
from pathlib import Path
from agents import Agent, Runner, function_tool
from agents.mcp import MCPServerStdio

excel_server_path = str(Path("../mcp_servers/excel_server.py").resolve())


async def excel_workflow():
    """Full workflow: read Excel → analyze → fix errors → save → report."""

    async with MCPServerStdio(
        name="Excel Tools",
        params={"command": sys.executable, "args": [excel_server_path]},
    ) as excel_server:

        # ---- Agent 1: Reader ----
        # Reads the Excel file and extracts the raw data
        reader_agent = Agent(
            name="Reader Agent",
            instructions=(
                "You read Excel files and present their contents clearly. "
                "When asked, read the file and show the full table data. "
                "Also provide column statistics when relevant."
            ),
            mcp_servers=[excel_server],
            model="gpt-4o-mini",
        )

        # ---- Agent 2: Analyst ----
        # Analyzes data, finds errors, suggests corrections
        analyst_agent = Agent(
            name="Analyst Agent",
            instructions=(
                "You are a data quality analyst. Given spreadsheet data, you:\n"
                "1. Check for data quality issues (zeros, missing values, outliers).\n"
                "2. Verify calculations (e.g., Total_Pay should equal Hours_Worked × Hourly_Rate + Bonus).\n"
                "3. List every error you find with the specific cell reference and the correct value.\n"
                "Be precise: provide exact cell references (e.g., F2) and correct values."
            ),
            mcp_servers=[excel_server],
            model="gpt-4o-mini",
        )

        # ---- Agent 3: Writer ----
        # Applies corrections to the Excel file
        writer_agent = Agent(
            name="Writer Agent",
            instructions=(
                "You update Excel files. When given a list of corrections (cell references and new values), "
                "apply each one using the update_cell tool. Confirm each update."
            ),
            mcp_servers=[excel_server],
            model="gpt-4o-mini",
        )

        # ---- Agent 4: Summary ----
        # Produces a human-readable summary of all changes
        summary_agent = Agent(
            name="Summary Agent",
            instructions=(
                "You write clear, professional summaries. Given a description of data issues found "
                "and corrections made, produce a concise summary report with:\n"
                "- Number of issues found\n"
                "- What each issue was\n"
                "- What was corrected\n"
                "- Final status"
            ),
            model="gpt-4o-mini",
        )

        # ---- Orchestrator ----
        orchestrator = Agent(
            name="Payroll Orchestrator",
            instructions=(
                "You manage a payroll data quality workflow. Follow these steps EXACTLY:\n\n"
                "1. Use the reader tool to read 'employee_payroll.xlsx' and get the full data.\n"
                "2. Use the analyst tool to analyze the data. Ask it to check that "
                "   Total_Pay = (Hours_Worked × Hourly_Rate) + Bonus for each row, "
                "   and flag any other issues like zero values.\n"
                "3. Use the writer tool to apply the corrections the analyst suggested.\n"
                "4. Use the reader tool again to verify the file was updated correctly.\n"
                "5. Use the summary tool to produce a final report of all changes made.\n\n"
                "Present the summary report as your final output."
            ),
            tools=[
                reader_agent.as_tool(
                    tool_name="reader",
                    tool_description="Read and display Excel file contents and statistics.",
                ),
                analyst_agent.as_tool(
                    tool_name="analyst",
                    tool_description="Analyze data for errors, verify calculations, suggest corrections with exact cell refs.",
                ),
                writer_agent.as_tool(
                    tool_name="writer",
                    tool_description="Apply corrections to cells in the Excel file.",
                ),
                summary_agent.as_tool(
                    tool_name="summarizer",
                    tool_description="Produce a clear summary report of changes made.",
                ),
            ],
            model="gpt-4o-mini",
        )

        # Run the full workflow
        print("🚀 Starting payroll data quality workflow...")
        print("   File: employee_payroll.xlsx")
        print()

        result = await Runner.run(
            orchestrator,
            "Please analyze the employee_payroll.xlsx file. Check all calculations, "
            "find any errors, fix them, and give me a summary of what was changed.",
        )

        print("\n" + "=" * 60)
        print("📊 WORKFLOW COMPLETE")
        print("=" * 60)
        print(result.final_output)

        return result


result = await excel_workflow()

### Step 4: Verify the Updated File

In [ ]:
import pandas as pd
from pathlib import Path

# Read the updated file
updated_df = pd.read_excel("/tmp/mcp_workdir/employee_payroll.xlsx")

print("📄 Updated Excel file contents:")
print()
print(updated_df.to_string(index=False))

# Verify calculations
print("\n\n🔍 Verification: Total_Pay == (Hours_Worked × Hourly_Rate) + Bonus")
print()
updated_df["Expected"] = (updated_df["Hours_Worked"] * updated_df["Hourly_Rate"]) + updated_df["Bonus"]
updated_df["Correct?"] = updated_df["Total_Pay"] == updated_df["Expected"]
print(updated_df[["Employee", "Total_Pay", "Expected", "Correct?"]].to_string(index=False))

### Step 5: Inspect the Full Conversation Trace

In [ ]:
from agents.items import ToolCallItem, ToolCallOutputItem, MessageOutputItem

print("=" * 60)
print("📋 WORKFLOW TRACE — All steps the orchestrator took")
print("=" * 60)

for i, item in enumerate(result.new_items, 1):
    if isinstance(item, ToolCallItem):
        args_preview = item.raw_item.arguments[:120] if item.raw_item.arguments else ""
        print(f"\n  [{i}] 🔧 Called: {item.raw_item.name}")
        print(f"      Input: {args_preview}...")
    elif isinstance(item, ToolCallOutputItem):
        output_preview = str(item.output)[:200]
        print(f"  [{i}] 📤 Result: {output_preview}...")
    elif isinstance(item, MessageOutputItem):
        text = ""
        if hasattr(item.raw_item, "content") and item.raw_item.content:
            for part in item.raw_item.content:
                if hasattr(part, "text"):
                    text += part.text
        if text:
            print(f"\n  [{i}] 💬 {item.raw_item.role}: {text[:200]}...")

---
## Recap: How the Excel Workflow Works

```
1. USER uploads employee_payroll.xlsx
   │
   ▼
2. ORCHESTRATOR calls → READER AGENT
   │                     │
   │                     └─ MCP call: read_excel("employee_payroll.xlsx")
   │                        Returns: full table data
   ▼
3. ORCHESTRATOR calls → ANALYST AGENT
   │                     │
   │                     └─ MCP calls: read_excel, get_cell_value, etc.
   │                        Returns: list of errors with cell refs + correct values
   ▼
4. ORCHESTRATOR calls → WRITER AGENT
   │                     │
   │                     └─ MCP calls: update_cell("F2", 12500), update_cell("F3", ...), ...
   │                        Returns: confirmation of each update
   ▼
5. ORCHESTRATOR calls → READER AGENT  (verification)
   │                     │
   │                     └─ MCP call: read_excel("employee_payroll.xlsx")
   │                        Returns: updated table data
   ▼
6. ORCHESTRATOR calls → SUMMARY AGENT
   │                     │
   │                     └─ Produces human-readable change report
   ▼
7. ORCHESTRATOR returns final summary to USER
   User can now download the corrected Excel file.
```

---
## 8 · Proposed Project Structure for Production

Here's how to structure a multi-agent + MCP project for production use:

```
excel-agent-project/
│
├── .env                              # OPENAI_API_KEY, MCP_WORK_DIR, etc.
├── .env.example
├── .gitignore
├── pyproject.toml                    # Dependencies: openai-agents, mcp[cli], openpyxl, etc.
├── README.md
│
├── mcp_servers/                      # ⬅ MCP servers are standalone programs
│   ├── excel_server.py              # Excel read/write/stats tools
│   ├── database_server.py           # DB query tools (optional)
│   └── file_server.py              # Generic file I/O tools
│
├── src/
│   ├── __init__.py
│   │
│   ├── agents/                      # Agent definitions
│   │   ├── __init__.py
│   │   ├── orchestrator.py          # Main orchestrator (coordinates sub-agents)
│   │   ├── reader_agent.py          # Reads files via MCP
│   │   ├── analyst_agent.py         # Analyzes data, finds errors
│   │   ├── writer_agent.py          # Writes corrections via MCP
│   │   └── summary_agent.py         # Produces reports
│   │
│   ├── tools/                       # Non-MCP function tools (inline helpers)
│   │   ├── __init__.py
│   │   ├── formatting.py            # format_currency, format_date, etc.
│   │   └── validation.py            # Data validation helpers
│   │
│   ├── mcp_connections/             # MCP server connection configs
│   │   ├── __init__.py
│   │   └── servers.py               # Factory functions to create MCPServerStdio instances
│   │
│   ├── workflows/                   # High-level workflow orchestration
│   │   ├── __init__.py
│   │   ├── excel_workflow.py        # The full Excel processing pipeline
│   │   └── report_workflow.py       # Report generation pipeline
│   │
│   ├── models/                      # Pydantic models
│   │   ├── __init__.py
│   │   └── schemas.py               # Shared data schemas
│   │
│   ├── config.py                    # Load .env, set defaults, model names
│   └── main.py                      # Entry point
│
├── uploads/                          # Where user-uploaded files land
│   └── .gitkeep
│
├── outputs/                          # Where processed files are saved
│   └── .gitkeep
│
├── notebooks/                        # Experimentation
│   └── mcp_tutorial.ipynb
│
└── tests/
    ├── test_mcp_servers.py           # Test MCP tools independently
    ├── test_agents.py               # Test agent behavior
    └── test_workflows.py            # End-to-end workflow tests
```

### Key design decisions

| Decision | Rationale |
|---|---|
| **MCP servers live in `mcp_servers/`** | They are standalone programs. You can test them independently, version them separately, or even deploy them as remote HTTP services later. |
| **`mcp_connections/servers.py`** | Factory functions that create `MCPServerStdio(...)` or `MCPServerStreamableHttp(...)` instances. Centralizes paths and configs. |
| **`workflows/` folder** | Each workflow is a function that wires up agents + MCP servers + runs them. Separates orchestration logic from agent definitions. |
| **`uploads/` and `outputs/`** | Clear separation of input vs. output files. The `MCP_WORK_DIR` env var points to one of these. |
| **Non-MCP tools in `tools/`** | Quick inline helpers that don't need a full MCP server (e.g., date formatting). |

### Example: `src/mcp_connections/servers.py`

```python
# src/mcp_connections/servers.py
import sys
from pathlib import Path
from agents.mcp import MCPServerStdio

MCP_DIR = Path(__file__).parent.parent.parent / "mcp_servers"

def get_excel_server():
    return MCPServerStdio(
        name="Excel Tools",
        params={
            "command": sys.executable,
            "args": [str(MCP_DIR / "excel_server.py")],
        },
        cache_tools_list=True,  # tools don't change at runtime
    )
```

### Example: `src/workflows/excel_workflow.py`

```python
# src/workflows/excel_workflow.py
from agents import Runner
from src.agents.orchestrator import build_orchestrator
from src.mcp_connections.servers import get_excel_server

async def process_excel(filepath: str, user_instruction: str) -> str:
    async with get_excel_server() as excel_server:
        orchestrator = build_orchestrator(excel_server)
        result = await Runner.run(
            orchestrator,
            f"Process the file '{filepath}'. {user_instruction}",
        )
        return result.final_output
```

### Going Remote: stdio → HTTP

When you're ready for production, you can switch from `MCPServerStdio` to `MCPServerStreamableHttp` with minimal changes:

```python
# Development (local subprocess)
server = MCPServerStdio(
    name="Excel Tools",
    params={"command": "python", "args": ["excel_server.py"]},
)

# Production (remote HTTP server)
from agents.mcp import MCPServerStreamableHttp

server = MCPServerStreamableHttp(
    name="Excel Tools",
    params={
        "url": "https://my-mcp-server.example.com/mcp",
        "headers": {"Authorization": f"Bearer {API_KEY}"},
    },
)
```

Your agent code stays **exactly the same** — only the server connection changes. That's the power of MCP standardization!

---

## Summary & Cheatsheet

| What | How |
|---|---|
| Build an MCP server | `from mcp.server.fastmcp import FastMCP; mcp = FastMCP("name")` |
| Define an MCP tool | `@mcp.tool()` decorator on a Python function |
| Run MCP server (stdio) | `mcp.run(transport="stdio")` |
| Connect agent to MCP (stdio) | `MCPServerStdio(name=..., params={"command": ..., "args": [...]})` |
| Connect agent to MCP (HTTP) | `MCPServerStreamableHttp(name=..., params={"url": ...})` |
| Hosted MCP (zero infra) | `HostedMCPTool(tool_config={"type": "mcp", "server_url": ...})` |
| Give MCP server to agent | `Agent(mcp_servers=[server])` |
| Combine MCP + function tools | `Agent(mcp_servers=[server], tools=[my_func_tool])` |
| Multiple MCP servers | `Agent(mcp_servers=[server1, server2])` |
| Cache tool list | `MCPServerStdio(..., cache_tools_list=True)` |
| Filter MCP tools | `MCPServerStdio(..., tool_filter=create_static_tool_filter(...))` |

### MCP vs Function Tools — When to use which?

| Use MCP tools when... | Use `@function_tool` when... |
|---|---|
| The tool should be reusable across projects | The tool is specific to this project |
| The tool needs its own process/resources | The logic is simple and stateless |
| You want language/framework independence | You want quick iteration in Python |
| You plan to deploy remotely | You're in notebook/prototype mode |
| You want to share tools across teams | You're the only consumer |

### Next steps
- 📖 [MCP Specification](https://modelcontextprotocol.io)
- 🔌 [OpenAI Agents + MCP Docs](https://openai.github.io/openai-agents-python/mcp/)
- 🧪 [MCP Examples (GitHub)](https://github.com/openai/openai-agents-python/tree/main/examples/mcp)
- 🛡️ [Tool Filtering & Approval Flows](https://openai.github.io/openai-agents-python/mcp/#tool-filtering)
- 🌐 [Hosted MCP (zero infra)](https://openai.github.io/openai-agents-python/mcp/#1-hosted-mcp-server-tools)